In [1]:
#获取数据
from keras.datasets import boston_housing
(train_data, train_targets),(test_data, test_targets) = boston_housing.load_data()


57344/57026 [==============================] - 2s 30us/step


In [5]:
train_data[0]


array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [6]:
#数据处理
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
#此处用的mean 和 std均为训练数据集计算而来。 不能使用测试数据集上得到的结果

test_data -= mean
test_data /= std




In [7]:
#定义模型
from keras import models
from keras import layers


def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                          input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,activation='relu',))
    model.add(layers.Dense(1))#，没有激活函数 只有一个单元 标量回归的设置 添加激活函数会限制输出范围 
    model.compile(optimizer='rmsprop',
                 loss='mse',
                 metrics=['mae'])#损失函数为mse 均方误差 预测至于目标值的差的平方   回归问题常用的损失函数
    return model 




In [17]:
#k折交叉验证   将数据划分K个分区 实例化k个模型 
#将每个模型在k-1分区训练 并在最后一个分区评估 验证分数为k个平均值    
#验证集数据小 验证分数波动很大 方差很大 无法对模型可靠评估

import numpy as np

k = 4
num_val_samples = len(train_data)//k
num_epochs = 500
scores = []
all_mae_history=[]
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples : (i+1)*num_val_samples]
    val_targets = train_targets[i*num_val_samples : (i+1)*num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i*num_val_samples],
        train_data[(i+1)*num_val_samples:]],
        axis = 0
    )
    
    partial_train_targets = np.concatenate(
        [train_targets[:i*num_val_samples],
        train_targets[(i+1)*num_val_samples:]],
        axis = 0
    )
    
    model = build_model()
    history = model.fit(partial_train_data,partial_train_targets,validation_data=(val_data,val_targets), epochs=num_epochs,batch_size=1,verbose=0)
    mae_history = history.history['val_mean_absolute_error']
    
    all_mae_history.append(mae_history)
    
#     val_mse, val_mae = model.evaluate(val_data,val_targets,verbose=0)
#     scores.append(val_mae)
    
    
        
        

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [11]:
all_mea


[2.0166878818285348, 2.2932473550928703, 2.9657923089395655, 2.326364535506409]

In [12]:
np.mean(scores)

2.400523020341845